# Docker
In this demo we are going to go through the process of utilizing Docker as part of the Model Deployment process.

We will cover Docker basics, build an image with our containing our flask app, push it to a repo, and finally run a container from the image.

## Installing Docker
You can install the Docker engine on Linux, Windows and Mac.

Below steps uses Ubuntu using `apt`.

### First we need to add the official GPG Key

```bash
apt-get update
apt-get install ca-certificates curl
install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
chmod a+r /etc/apt/keyrings/docker.asc
```

### Next we need to add the repository to Apt sources.

```bash
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  
  tee /etc/apt/sources.list.d/docker.list > /dev/null
```


### Run the installation of the CLI and the Engine

```bash
apt-get update
apt-get install docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin -y
```

### Start and enable the service
```bash
systemctl start docker.service
systemctl enable docker.service
```

### Verify the installation
```bash
docker version
docker run hello-world
```

## Building a Docker image
Building a Docker image involves using the `docker build` command to package an application and its dependencies into a portable template called an image. 

This process uses a Dockerfile, which are instructions to define the image’s contents and configuration.

In the same path as the Dockerfile.

```bash 
docker build -t mobilenetv3lg-flask:v1.0 .
```

NOTE: the size of the image on initial build.

## Creating a Dockerfile
Creating a Dockerfile involves writing a simple text file that defines the steps to package an application into a Docker image. 

It includes instructions such as the base image to use, dependencies to install, files to include, and the commands to run the application.

```bash
# Use official Python base image
FROM python:3.11-slim

# Set the working directory
WORKDIR /opt/app

# Copy requirements and install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy app code
COPY ./flask_app .

# Create a user and group for running the app
RUN groupadd -r pytorch && useradd --no-log-init -r -g pytorch pytorch

# Change ownership of the home directory
RUN chown -R pytorch:pytorch /home/pytorch

# Change ownership of the app directory
RUN chown -R pytorch:pytorch /opt/app

# Switch to the created user
USER pytorch

# Expose the port for Flask
EXPOSE 8000

# Command to run the app
CMD ["flask", "run", "--host=0.0.0.0", "--port=8000"]

```

 ## Running a Container from an Image
Once we have our model built we can run a container from our model using the `docker run` command.

```bash
docker run -p 8000:8000 mobilenetv3lg-flask:v1.0
```

NOTE: add a `-d` command makes it run in daemon mode where it starts and runs until killed but docker CLI.

In [ ]:
# Test a request
import requests
import base64

# Open image and encode it
with open('dog-1.jpg', 'rb') as img_file:
    base64_string = base64.b64encode(img_file.read()).decode('utf-8')

# JSON payload with the Base64 encoded image
payload = {
    "image": base64_string
}

# Set the headers
headers = {
    "Content-Type": "application/json"
}

# Send POST request
response = requests.post("http://127.0.0.1:8000/predict", 
                         json=payload, 
                         headers=headers)

# Print the response
print("Response JSON:", response.json())

In [ ]:
import json 

# Downloaded from Hugging Face
# https://huggingface.co/datasets/huggingface/label-files/blob/main/imagenet-1k-id2label.json
with open("labels.json", "r") as f:
    imagenet_classes = json.load(f)

# Get the actual class name from our labels
print(imagenet_classes['207'])

## Push a Docker Image to a Registry
Once you have tested your image, you can push it to a registry for others to use. 

Pushing an image to a Docker registry involves tagging the image with the registry name and using the docker push command to upload it.

```bash
docker tag mobilenetv3lg-flask:v1.0 username/mobilenetv3lg-flask:v1.0
```

Then push it.
```bash
docker push username/mobilenetv3lg-flask:v1.0
```

NOTE: You must be authenticated before logging in. You can use `docker login` command for docker registry.

## Pulling an Image
Once the image lives on a registry, anyone with access can pull the image and create a container from it.

```bash
docker pull username/mobilenetv3lg-flask:v1.0
```

```bash
docker run -p 8000:8000 username/mobilenetv3lg-flask:v1.0
```